# Exercício 2 (Correção Bubble Sort) - Trabalho Prático 4

**Grupo 4:**<br>Carlos  Costa-A94543<br>Ruben Silva-A94633

# Problema: 
O programa Python seguinte implementa o algoritmo de bubble sort para ordenação in situ de um array de inteiros seq.


    seq = [-2,1,2,-1,4,-4,-3,3]
    changed = True
    while changed:
        changed = False
        for i in range(len(seq) - 1):
            if seq[i] > seq[i+1]:
                seq[i], seq[i+1] = seq[i+1], seq[i]
                changed = True
    pass     


1. Defina a pré-condição e a pós-condição que descrevem a especificação deste algoritmo.
2. Sugira um invariante.
3. O ciclo for pode ser descrito por uma transição $\,\mathtt{seq}\gets \mathit{exp}(\mathtt{seq})\,$. Construa uma relação de transição $\;\mathsf{trans}(\mathtt{seq},\mathtt{seq}')\,$ que modele esta atribuição.
4. Usando a técnica que lhe parecer mais conveniente verifique a correção do algoritmo.

# Análise do Problema
Este é um problema sobre um a correção de um algoritmo, neste caso especificio, um BubbleSort.<br>
Para o resolução teorica do problema foi usada as seguintes variáveis:

* $seq \rightarrow$ Array
* $n ∈ \mathbb{N} \rightarrow$ Indice atual do array


# Condições do trans para o trans(seq,seq′)
1. **same_list**
$$\forall_{n}:seq_n′ = seq_n$$ 
2. **trans01**
$$seq_n > seq_{n+1} ∧ seq_n′ = seq_{n+1} ∧ seq_{n+1}′ = seq_n ∧ samelist$$ 
3. **trans02**
$$seq_n <= seq_{n+1} ∧ seq_n′ = seq_{n} ∧ seq_{n+1}′ = seq_{n+1} ∧ samelist$$

# Pré e Pós Condições 
1. Pré-Condição:<br>
Esta pré condição garante-nos que a lista tem o minímo de elementos para ser possível fazer uma comparação
$$ n >=2$$
2. Pós-Condição:<br>
Esta pré condição garante-nos que a cada membro da lista é menor que o seu sucessor
$$\forall_{i < n}: seq[i] \le seq[i+1]$$

# Codificação do Algoritmo

**Importar o solver**
1. Importar o z3-solver

In [1]:
from z3 import *

**Função "declare"**<br>
Esta função é responsável pela declaração de todas as variáveis que serão utilizadas no solver.
1. Parâmetros:
    1. $i$ -> um inteiro que será responsável por dar o nr às variaveis
    2. $size$ -> Tamanho do array
2. Função:
    1. Inicialmente criamos um dicionário para colocar todas as variáveis necessárias.
    2. Criamos $n$ variáveis: (todos os membros do array)
3. Return do novo dicionário com as variáveis

In [2]:
def declare(i, size):
    state = {}
    for n in range(size):
        state["seq"+str(n)] = Int("seq"+str(n)+"frame"+str(i))
    return state

**Função "init"**<br>
Esta função é responsável pela inicialização do primeiro node do traço.
1. Parâmetros:
    1. $state$ -> Primeiro node do traço
    2. $sequence$ -> Array para usar no FOTS
2. Return de um "And" com a seguinte condição lógica: todos os membros do traço inicial com o mesmo valor do $sequence$

In [3]:
def init(state, sequence):
    init_list = []
    for n in range(len(sequence)):
        init_list.append(state["seq"+str(n)] == sequence[n])
    return And(init_list)

**Função "trans"**<br>
Esta função é responsável pela criação das conexões lógicas necessárias para o FOTS.
1. Parâmetros:
    1. $curr$ -> Membro atual do traço
    2. $prox$ -> Membro seguinte ao atual do traço
    3. $n$ -> Index atual do array
    4. $size$ -> Tamanho do array
2. Função:
    1. Criamos as condições lógicas:
        1. trans01: $(seq_n > seq_{n+1} ∧ seq_n′ = seq_{n+1} ∧ seq_{n+1}′ = seq_n ∧ samelist)$
        2. trans02: $(seq_n <= seq_{n+1} ∧ seq_n′ = seq_{n} ∧ seq_{n+1}′ = seq_{n+1} ∧ samelist)$
3. Return de um "Or" com a seguinte condição lógica:  ($ trans01 \vee trans02$)

In [4]:
def trans(curr, prox, n, size):
    same_list = []
    if (n+1!=size):
        for i in range (size):
            if i != n and i != n+1:
                same_list.append(prox["seq"+str(i)] == curr["seq"+str(i)])


        trans01 = And(curr["seq"+str(n)] > curr["seq"+str(n+1)],
                      prox["seq"+str(n)] == curr["seq"+str(n+1)],
                      prox["seq"+str(n+1)] == curr["seq"+str(n)],
                      And(same_list))

        trans02 = And(curr["seq"+str(n)] <= curr["seq"+str(n+1)],
                      prox["seq"+str(n)] == curr["seq"+str(n)],
                      prox["seq"+str(n+1)] == curr["seq"+str(n+1)],
                      And(same_list))

        

        return Or(trans01,trans02)

    for i in range (size):
        same_list.append(prox["seq"+str(i)] == curr["seq"+str(i)])
    return And(same_list)

**Função "traço"**<br>
Esta é a função principal e é a que irá juntar as funções todas e gerar o traço pretendido e com ele tabelar o output
1. Parâmetros:
    1. $declare$ -> Função declare
    2. $init$ -> Função init
    3. $trans$ -> Função trans
    4. $sequence$ -> Array para usar no FOTS
2. Função:
    1. Iniciamos o Solver
    2. Calculamos o tamanho do traço
    3. Criar o traço com as variáveis
    4. Inicializar as variáveis essenciais no primeiro node do traço
    5. Criar a conexão lógica entre os nodes do traço todos e adicionar ao solver
    6. Correr o Solver e tabelar o resultado

In [5]:
def traco(declare, init, trans, sequence):
    s = Solver()
    size = len(sequence)
    k = size*size

    #Gerar o traço
    trace = [declare(i, size) for i in range(k)]

    #Gerar o primeiro estado
    s.add(init(trace[0], sequence))

    #Gerar as condições lógicas do traço
    for i in range(k-1):
        s.add(trans(trace[i], trace[i+1], i%size, size))

    old_sequence =[]
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            sequence_var= []
            for v in trace[i]:
                sequence_var.append(m[trace[i][v]])
            if i%size==0:
                if old_sequence == sequence_var :
                    return
                print("Frame: "+ str(i)) 
                print(sequence_var)
                old_sequence=sequence_var


# Correção do Algoritmo

**Pré/Pós-condição**


In [6]:
def pre(sequence):
    return And(len(sequence)>=2)

def pos(curr,sequence):
    return And([curr["seq"+str(i)]<=curr["seq"+str(i+1)] for i in range(len(sequence)-2)])


### Strongest post-condition

Na abordagem SPC a denotação de um fluxo com um comando de atribuição introduz um quantificador existencial, o que não é adequado à verificação com SMT solvers: 
$ \quad [ C \; ; x = e ] \; =  \; \exists a. (x = e[a/x]) \wedge [C][a/x] $

Para lidar com este problema pode-se converter o programa original ao formato "*single assignment*" (SA).
Num programa SA cada variável só pode ser usada depois de ser atribuida e só pode ser atribuída uma única vez.

Um programa (onde variáveis são atribuídas mais do que uma vez) pode ser reescrito num programa SA criando "clones" distintos das variáveis de forma a que seja possível fazer uma atribuição única a cada instância.

Neste caso, a denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica SPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = \phi \\
[{\sf assert}\:\phi] = \phi \\
[x = e ] = (x = e)\\
[(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
[C \, ; {\sf skip}\;] = [C] \\
[C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
[C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
[ C \, ; x = e ] = [C] \wedge (x = e)\\
[C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)]
\end{array}
$



In [7]:
def strongestPostCondition(declare, init, trans, sequence):
    s = Solver()
    size = len(sequence)
    k = size*size

    #Pre-Condição
    pre_Condition = pre(sequence)

    #Gerar o traço
    trace = [declare(i, size) for i in range(k)]

    #Variavel para fazer as condições logicas do programa
    prog = []

    #Gerar o primeiro estado
    prog.append(And(init(trace[0], sequence)))

    #Gerar as condições lógicas do traço
    for i in range(k-1):
        prog.append(And(trans(trace[i], trace[i+1], i%size, size)))

    #Pos-Condição
    pos_Condition = pos(trace[len(prog)-1], sequence)

    
    #Formula final do strongest post condition
    #Transformar a lista numa sequencia lógica
    prog = And(prog)

    #Funfção final do SPC
    spc = Not(Implies(And(pre_Condition,prog), pos_Condition))

    #Adicionar o spc ao solver
    s.add(spc)

    #Correr
    if s.check() == sat:
        print("Falhou")
    print("Correção do Algoritmo provado com sucesso!!!\n")

**Executar o código**

In [8]:
sequence = [-2,1,2,-1,4,-4,-3,3]
strongestPostCondition(declare, init, trans, sequence)
traco(declare, init, trans, sequence)


Correção do Algoritmo provado com sucesso

Frame: 0
[-2, 1, 2, -1, 4, -4, -3, 3]
Frame: 8
[-2, 1, -1, 2, -4, -3, 3, 4]
Frame: 16
[-2, -1, 1, -4, -3, 2, 3, 4]
Frame: 24
[-2, -1, -4, -3, 1, 2, 3, 4]
Frame: 32
[-2, -4, -3, -1, 1, 2, 3, 4]
Frame: 40
[-4, -3, -2, -1, 1, 2, 3, 4]
